In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dense
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import os
from scipy import stats
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]= "true"

2023-05-31 13:12:42.738932: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
def create_model():
    model = Sequential()
    model.add(tf.keras.layers.Conv1D(256, 5, padding="same", activation="relu", input_shape=(14,256))) # tf.keras.layers.Conv1D 
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense (512,activation="relu"))
    #model.add(tf.keras.layers.Dense (4,activation="softmax"))
    model.add(tf.keras.layers.Dense (2,activation="softmax"))
    
    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer="adam", metrics=['accuracy'])  # 損失関数はbinary_crossentropy
    return model

In [11]:
# データの読み込み
df = pd.read_csv('Raw.csv')
#df = pd.read_csv('emotion.csv')

# 1秒ごとのデータに分割
ch_count = 14
n_samples_per_second = 256  # 256Hzのサンプリングレート

total_seconds = 10  # 全体の秒数
#total_seconds = 3242  # 全体の秒数

# ラベルの作成（5秒閉じ、5秒開け）
labels_close = [0] * 5 
labels_open = [1] * 5 
labels = labels_close + labels_open
#labels = np.array([df["State"][i*256] for i in range(total_seconds)])
labels = to_categorical(labels)

# データとラベルを1秒ごとに分割
X = np.array([df.iloc[i*n_samples_per_second:(i+1)*n_samples_per_second, :-1].values for i in range(total_seconds)])

X = X.transpose(2,0,1)
for i in range(ch_count):
    scaler = StandardScaler()
    X[i] = scaler.fit_transform(X[i])
X = X.transpose(1,0,2)

# 学習用とテスト用に分割
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=0)
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=0)

# モデル作成
model = create_model()

# 学習
#model.fit(X_train, y_train, epochs=100, batch_size=16, verbose=1, validation_data = [X_val, y_val], callbacks=[tf.keras.callbacks.EarlyStopping(patience=20)])
model.fit(X_train, y_train, epochs=100, batch_size=3, verbose=1)

# 訓練データに対する評価
train_scores = model.evaluate(X_train, y_train, verbose=1)
print(f"Training Accuracy: {train_scores[1]*100}")

# テストデータに対する評価
test_scores = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Accuracy: {test_scores[1]*100}")

Epoch 1/100
3/3 [==============================] - 1s 9ms/step - loss: 0.8809 - accuracy: 0.6250
Epoch 2/100
3/3 [==============================] - 0s 3ms/step - loss: 1.1325e-06 - accuracy: 1.0000
Epoch 3/100
3/3 [==============================] - 0s 2ms/step - loss: 5.8114e-07 - accuracy: 1.0000
Epoch 4/100
3/3 [==============================] - 0s 2ms/step - loss: 4.1574e-06 - accuracy: 1.0000
Epoch 5/100
3/3 [==============================] - 0s 2ms/step - loss: 1.3411e-06 - accuracy: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 2ms/step - loss: 3.8743e-07 - accuracy: 1.0000
Epoch 7/100
3/3 [==============================] - 0s 2ms/step - loss: 1.3411e-07 - accuracy: 1.0000
Epoch 8/100
3/3 [==============================] - 0s 2ms/step - loss: 2.9802e-08 - accuracy: 1.0000
Epoch 9/100
3/3 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 10/100
3/3 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 

3/3 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 82/100
3/3 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 83/100
3/3 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 84/100
3/3 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 85/100
3/3 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 86/100
3/3 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 87/100
3/3 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 88/100
3/3 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 89/100
3/3 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 90/100
3/3 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 

In [24]:

def create_model():
    model = Sequential()
    model.add(tf.keras.layers.Dense(64, activation="relu", input_shape=(14,)))
    model.add(tf.keras.layers.Dense(32,activation="relu"))
    model.add(tf.keras.layers.Dense(2,activation="softmax"))
    
    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer="adam", metrics=['accuracy'])  # 損失関数はbinary_crossentropy
    return model


In [ ]:
"""
df = pd.read_csv('emotion.csv')
y = df["State"]
df = df.drop(["State"],axis=1)
"""

In [26]:
"""
X = np.array(df).transpose(1,0).reshape(14,-1,1)
for i in range(ch_count):
    scaler = StandardScaler()
    X[i] = scaler.fit_transform(X[i].reshape(-1,1))
X = X.transpose(1,0,2).reshape(-1,14)

y = to_categorical(y)
"""

# データの読み込み
df = pd.read_csv('Raw.csv')

# チャンネルごとに正規化
scaler = StandardScaler()
for column in df.columns[:-1]:  # 'State'列を除くすべての列
    df[column] = scaler.fit_transform(df[column].values.reshape(-1, 1))

X = df.drop('State', axis=1).values
y = df['State'].values
y = to_categorical(y)

# 学習用とテスト用に分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=0)

# モデル作成
model = create_model()

# 学習
model.fit(X_train, y_train, epochs=100, batch_size=3, verbose=1)
#model.fit(X_train, y_train, epochs=10000, batch_size=64, verbose=1, validation_data = [X_val, y_val], callbacks=[tf.keras.callbacks.EarlyStopping(patience=50)])

# 訓練データに対する評価
train_scores = model.evaluate(X_train, y_train, verbose=1)
print(f"Training Accuracy: {train_scores[1]*100}")

# テストデータに対する評価
test_scores = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Accuracy: {test_scores[1]*100}")

Epoch 1/100
683/683 [==============================] - 1s 1ms/step - loss: 0.1749 - accuracy: 0.9395
Epoch 2/100
683/683 [==============================] - 1s 1ms/step - loss: 0.0191 - accuracy: 0.9956
Epoch 3/100
683/683 [==============================] - 1s 1ms/step - loss: 0.0068 - accuracy: 0.9985
Epoch 4/100
683/683 [==============================] - 2s 3ms/step - loss: 0.0055 - accuracy: 0.9985
Epoch 5/100
683/683 [==============================] - 2s 3ms/step - loss: 0.0068 - accuracy: 0.9990
Epoch 6/100
683/683 [==============================] - 1s 1ms/step - loss: 0.0022 - accuracy: 0.9995
Epoch 7/100
683/683 [==============================] - 1s 1ms/step - loss: 4.6375e-04 - accuracy: 1.0000
Epoch 8/100
683/683 [==============================] - 1s 1ms/step - loss: 4.9896e-04 - accuracy: 1.0000
Epoch 9/100
683/683 [==============================] - 1s 1ms/step - loss: 1.9798e-04 - accuracy: 1.0000
Epoch 10/100
683/683 [==============================] - 1s 2ms/step - loss: 1.4

683/683 [==============================] - 2s 2ms/step - loss: 8.8578e-05 - accuracy: 1.0000
Epoch 79/100
683/683 [==============================] - 2s 3ms/step - loss: 4.5751e-07 - accuracy: 1.0000
Epoch 80/100
683/683 [==============================] - 2s 2ms/step - loss: 4.1957e-07 - accuracy: 1.0000
Epoch 81/100
683/683 [==============================] - 2s 3ms/step - loss: 3.7383e-07 - accuracy: 1.0000
Epoch 82/100
683/683 [==============================] - 1s 1ms/step - loss: 3.2576e-07 - accuracy: 1.0000
Epoch 83/100
683/683 [==============================] - 1s 1ms/step - loss: 2.8461e-07 - accuracy: 1.0000
Epoch 84/100
683/683 [==============================] - 1s 1ms/step - loss: 2.4405e-07 - accuracy: 1.0000
Epoch 85/100
683/683 [==============================] - 1s 1ms/step - loss: 2.0284e-07 - accuracy: 1.0000
Epoch 86/100
683/683 [==============================] - 1s 1ms/step - loss: 1.5931e-07 - accuracy: 1.0000
Epoch 87/100
683/683 [==============================] - 1s 